In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import ltrim,rtrim,trim,col
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("MyFirstCSV").getOrCreate()


In [3]:
oij_df = spark.read.csv( path="OIJ.csv", sep=",", header=True,quote='"',inferSchema=True,)
inec_df = spark.read.csv( path="INEC.csv", sep=";", header=True,quote='"',inferSchema=True,)

In [ ]:
def remove_spaces(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito' or col == 'Provincia, cantón y distrito':
            df = df.withColumn(col, trim(col))
    return df


def to_lower_case(df):
    for col in df.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito' or col == 'Provincia, cantón y distrito':
            df = df.withColumn(col, lower(col))
    return df


oij_df = remove_spaces(oij_df)
inec_df = remove_spaces(inec_df)

oij_df = to_lower_case(oij_df)
inec_df = to_lower_case(inec_df)

oij_df.show(5)
inec_df.show(5)

In [ ]:
def find_non_matches(df1,df2):
    non_matches = []
    for col in df1.columns:
        if  col == 'Provincia' or col == 'Canton' or col == 'Distrito':
            for row in df1.select(col).distinct().collect():
                if not df2.filter(df2['Provincia, cantón y distrito'] == row[col]).collect():
                    non_matches.append(row[col])
    return non_matches

non_matches_oij = find_non_matches(oij_df,inec_df)

for non_match in non_matches_oij:
    print(non_match)
